In [ ]:
!pip install mido
!apt-get install -y fluidsynth
!pip install pyFluidSynth==1.3.0 pretty_midi soundfile


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fluidsynth is already the newest version (2.2.5-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
notes = ["도", "레", "미", "파", "솔", "라", "시"]
note2idx = {n: i for i, n in enumerate(notes)}
idx2note = {i: n for i, n in enumerate(notes)}

# 반짝반짝 작은별 앞부분
twinkle = [
    "도","도","솔","솔","라","라","솔",
    "파","파","미","미","레","레","도"
]

# 데이터 늘리기
melody = twinkle * 50
data = [note2idx[n] for n in melody]

seq_len = 4
X, Y = [], []
for i in range(len(data) - seq_len):
    X.append(data[i:i+seq_len])
    Y.append(data[i+seq_len])

import torch
X = torch.tensor(X, dtype=torch.long)
Y = torch.tensor(Y, dtype=torch.long)



In [ ]:
import torch.nn as nn

class MusicLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=1):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1])  # 마지막 시점만 사용
        return out, hidden



In [ ]:
model = MusicLSTM(vocab_size=len(notes), hidden_size=128)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    output, _ = model(X)
    loss = loss_fn(output, Y)
    loss.backward()
    optimizer.step()
    if epoch % 50 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f}")



Epoch 0 | Loss: 1.9445
Epoch 50 | Loss: 0.0001
Epoch 100 | Loss: 0.0001
Epoch 150 | Loss: 0.0000


In [ ]:
def generate_music(model, start_seq, length):
    model.eval()
    input_seq = torch.tensor([note2idx[n] for n in start_seq], dtype=torch.long).unsqueeze(0)
    generated = list(start_seq)
    hidden = None
    for _ in range(length):
        output, hidden = model(input_seq, hidden)
        probs = nn.functional.softmax(output, dim=-1).squeeze()
        next_idx = torch.multinomial(probs, 1).item()
        next_note = idx2note[next_idx]
        generated.append(next_note)
        input_seq = torch.cat([input_seq[:, 1:], torch.tensor([[next_idx]])], dim=1)
    return generated

generated_melody = generate_music(model, ["도","도","솔","솔"], length=30)
print("생성된 멜로디:", generated_melody)



생성된 멜로디: ['도', '도', '솔', '솔', '라', '라', '라', '솔', '파', '파', '파', '미', '미', '미', '레', '레', '레', '레', '레', '도', '도', '도', '도', '도', '도', '도', '도', '도', '솔', '솔', '솔', '라', '라', '솔']


In [ ]:
from mido import Message, MidiFile, MidiTrack

def save_midi(note_sequence, filename="twinkle_generated.mid", tempo=480):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    note_to_midi = {"도":60, "레":62, "미":64, "파":65, "솔":67, "라":69, "시":71}

    for n in note_sequence:
        midi_note = note_to_midi[n]
        track.append(Message("note_on", note=midi_note, velocity=64, time=0))
        track.append(Message("note_off", note=midi_note, velocity=64, time=tempo))

    mid.save(filename)
    print(f"MIDI 파일 저장 완료: {filename}")

save_midi(generated_melody)



MIDI 파일 저장 완료: twinkle_generated.mid


In [ ]:
import pretty_midi
import soundfile as sf

# 1. MIDI 불러오기
midi_data = pretty_midi.PrettyMIDI("twinkle_generated.mid")

# 2. 악기 지정 (피아노)
for inst in midi_data.instruments:
    inst.program = pretty_midi.instrument_name_to_program("Acoustic Grand Piano")

# 3. SoundFont 경로 (Colab에 업로드한 .sf2 파일, 예: FluidR3_GM.sf2)
sf2_path = "Yamaha_C3_Grand_Piano.sf2"

# 4. MIDI → 오디오 변환 (numpy array)
audio = midi_data.fluidsynth(sf2_path=sf2_path)

# 5. WAV 저장 (44100 Hz)
sf.write("twinkle_generated.wav", audio, 44100)
print("WAV 변환 완료!")

# 6. Colab에서 다운로드
from google.colab import files
files.download("twinkle_generated.wav")


WAV 변환 완료!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>